# Artificial Neural Network

### Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor


In [2]:
tf.__version__

'2.9.1'

## Part 1 - Data Preprocessing

### Importing the dataset

In [3]:
dataset = pd.read_excel('Dataset3.xlsx')
dataset.info()

#X=X.to_list()

#y=y.to_list()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_type        546 non-null    object 
 1   Charge_size        546 non-null    float64
 2   Standoff_distance  546 non-null    float64
 3   Incident_pressure  546 non-null    float64
dtypes: float64(3), object(1)
memory usage: 17.2+ KB


In [4]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Charge_type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_size        546 non-null    float64
 1   Standoff_distance  546 non-null    float64
 2   Incident_pressure  546 non-null    float64
 3   Charge_type_CompB  546 non-null    uint8  
 4   Charge_type_TNT    546 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 14.0 KB


In [5]:
dataset.head()

,Charge_size,Standoff_distance,Incident_pressure,Charge_type_CompB,Charge_type_TNT
0,0.5,1.5,283.258,0,1
1,0.5,2.5,163.904,0,1
2,0.5,3.5,135.678,0,1
3,0.5,4.5,124.039,0,1
4,0.5,5.5,117.856,0,1


In [6]:
y = dataset['Incident_pressure']
X = dataset.drop('Incident_pressure', axis=1)
print(X.shape, y.shape)

(546, 4) (546,)


In [7]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#X_train=np.asarray(X_train).astype(np.int)

#y_train=np.asarray(y_train).astype(np.int)

### Hyperparameter tuning - batch size, epoch

In [11]:
# Function to create model, required for KerasClassifier
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=100, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [12]:
# create model
model = KerasRegressor(model=create_model, verbose=0)

# define the grid search parameters
batch_size = [50, 60, 80, 100]
epochs = [50, 100]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.899693 using {'batch_size': 50, 'epochs': 100}
0.606975 (0.058195) with: {'batch_size': 50, 'epochs': 50}
0.899693 (0.036065) with: {'batch_size': 50, 'epochs': 100}
0.519128 (0.064557) with: {'batch_size': 60, 'epochs': 50}
0.886910 (0.035174) with: {'batch_size': 60, 'epochs': 100}
0.420370 (0.033439) with: {'batch_size': 80, 'epochs': 50}
0.780197 (0.018335) with: {'batch_size': 80, 'epochs': 100}
0.230609 (0.060922) with: {'batch_size': 100, 'epochs': 50}
0.671310 (0.059418) with: {'batch_size': 100, 'epochs': 100}


### Hyperparameter tuning - batch size, epoch, optimizer

In [8]:
# Function to create model, required for KerasClassifier
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=100, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [9]:
# create model
model = KerasRegressor(model=create_model, verbose=0)

# define the grid search parameters
batch_size = [50, 60, 80, 100]
epochs = [50, 100]
optimizer = ['RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.914535 using {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Nadam'}
0.496583 (0.063779) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'RMSprop'}
-0.096527 (0.037340) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'Adagrad'}
-0.177245 (0.033484) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'Adadelta'}
0.565627 (0.041256) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'Adam'}
0.093906 (0.024753) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'Adamax'}
0.647086 (0.014402) with: {'batch_size': 50, 'epochs': 50, 'model__optimizer': 'Nadam'}
0.827143 (0.058647) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'RMSprop'}
-0.029085 (0.027905) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adagrad'}
-0.174960 (0.035933) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adadelta'}
0.909620 (0.031926) with: {'batch_size': 50, 'epochs': 100, 'model__optimizer': 'Adam'}
0.390663 (0.036

### Hyperparameter tuning - batch size, epoch, learning rate

In [10]:
# Function to create model, required for KerasClassifier
def create_model():
 # create model
 model = Sequential()
 model.add(Dense(units=200, input_shape=(X_train.shape[1],), activation='relu'))
 model.add(Dense(units=100, activation='relu'))
 model.add(Dense(units=50, activation='relu'))
 model.add(Dense(units=1, activation='linear'))
 return model

In [ ]:
# create model
model = KerasRegressor(model=create_model, verbose=0, optimizer = 'Nadam', loss = 'mean_squared_error', metrics = ['mae'])

# define the grid search parameters
batch_size = [50, 60, 80, 100]
epochs = [50, 100]
learn_rate = [0.01, 0.1, 0.2]
#momentum = [0.8, 0.9]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.997277 using {'batch_size': 60, 'epochs': 100, 'optimizer__learning_rate': 0.01}
0.976974 (0.017614) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.01}
0.934511 (0.048929) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.1}
0.289896 (0.414893) with: {'batch_size': 50, 'epochs': 50, 'optimizer__learning_rate': 0.2}
0.996621 (0.002431) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.01}
0.880614 (0.097619) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.1}
0.299880 (0.426459) with: {'batch_size': 50, 'epochs': 100, 'optimizer__learning_rate': 0.2}
0.992978 (0.004278) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.01}
0.970004 (0.009733) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.1}
0.298315 (0.438947) with: {'batch_size': 60, 'epochs': 50, 'optimizer__learning_rate': 0.2}
0.997277 (0.001802) with: {'batch_size': 60, 'epochs': 100, 'optimizer__learn